In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Dec 12 22:27:52 2014

@author: LukasHalim
Forked by @edridgedsouza

2) For each post, determine earliest failure
- Sort the comments for each post in ascending order
- Create a variable for the comment number within each post
- In cases where there is a failure, identify the first failing comment
- In cases where there is not a failure, identify the final comment for the post


"""

import pandas as pd
import matplotlib.pyplot as plt
#import csv

import sqlite3
from godwin import Scraper, Database

db = Database('Godwin.db')
# db.reset_db()
s = Scraper(db)

In [ ]:
s.scrape_top_subreddits()

Scraping from /r/politicalcompassmemes: 100it [00:02, 48.19it/s]
Scraped 1 of 124 subreddits
Scraping from /r/europe: 100it [00:02, 49.26it/s]
Scraped 2 of 124 subreddits
Scraping from /r/teenagers: 100it [00:02, 46.35it/s]
Scraped 3 of 124 subreddits
Scraping from /r/witcher: 100it [00:01, 53.41it/s]
Scraped 4 of 124 subreddits
Scraping from /r/mechanicalkeyboards: 100it [00:01, 53.79it/s]
Scraped 5 of 124 subreddits
Scraping from /r/instagram: 100it [00:01, 86.23it/s]
Scraped 6 of 124 subreddits
Scraping from /r/moviedetails: 100it [00:01, 58.40it/s]
Scraped 7 of 124 subreddits
Scraping from /r/fo76: 100it [00:01, 50.69it/s]
Scraped 8 of 124 subreddits
Scraping from /r/whatcouldgowrong: 100it [00:02, 36.92it/s]
Scraped 9 of 124 subreddits
Scraping from /r/totalwar: 100it [00:01, 58.70it/s]
Scraped 10 of 124 subreddits
Scraping from /r/fortnitebr: 100it [00:02, 38.53it/s]
Scraped 11 of 124 subreddits
Scraping from /r/cringe: 100it [00:01, 97.78it/s]
Scraped 12 of 124 subreddits
Scrapi

Subreddit cringetopia forbidden


Scraping from /r/facepalm: 100it [02:59,  1.79s/it]
Scraped 15 of 124 subreddits
Scraping from /r/books: 100it [02:34,  1.54s/it]
Scraped 16 of 124 subreddits
Scraping from /r/pokemon: 100it [02:47,  1.68s/it]
Scraped 17 of 124 subreddits
Scraping from /r/apexlegends: 100it [02:45,  1.65s/it]
Scraped 18 of 124 subreddits
Scraping from /r/dndnext: 100it [02:47,  1.68s/it]
Scraped 19 of 124 subreddits
Scraping from /r/trashy: 0it [00:00, ?it/s]
Scraped 20 of 124 subreddits


Subreddit trashy forbidden


Scraping from /r/sysadmin: 100it [02:30,  1.51s/it]
Scraped 21 of 124 subreddits
Scraping from /r/insanepeoplefacebook: 100it [02:33,  1.53s/it]
Scraped 22 of 124 subreddits
Scraping from /r/apple: 100it [02:22,  1.43s/it]
Scraped 23 of 124 subreddits
Scraping from /r/dataisbeautiful: 100it [02:37,  1.57s/it]
Scraped 24 of 124 subreddits
Scraping from /r/manga: 100it [02:21,  1.41s/it]
Scraped 25 of 124 subreddits
Scraping from /r/relationships: 100it [02:12,  1.32s/it]
Scraped 26 of 124 subreddits
Scraping from /r/iama: 88it [01:48,  1.23s/it]
Scraped 27 of 124 subreddits
Scraping from /r/valorant: 100it [02:21,  1.41s/it]
Scraped 28 of 124 subreddits
Scraping from /r/unexpected: 100it [02:38,  1.59s/it]
Scraped 29 of 124 subreddits
Scraping from /r/askwomen: 63it [01:30,  1.34s/it]

In [59]:
import requests
x = requests.get('https://api.pushshift.io/reddit/comment/search',
            params = {'link_id':'knnflb', 
                      'q':['keyboard OR keyboards'], # How to make inclusive????
                      'size': 1, 'sort': 'asc'}) # Gives first result
x.raise_for_status()
x = x.json()['data']  # reverse so that oldest entries are first

In [52]:
y = requests.get('https://api.pushshift.io/reddit/submission/comment_ids/knnflb')
y.raise_for_status()
y = y.json()['data']  # Most recent entries are first here. So simply do y.index(x[0])

In [ ]:

comments_df = pd.read_sql("select * from comment",conn)
g = comments_df.groupby('post_id')
#Sort the comments for each post in ascending order
comments_df['RN'] = g['comment_created'].rank(method='first')
#Create a variable for the comment number within each post
comments_with_nazi_df = comments_df[comments_df.nazi_in_comment == 1]
#Identify posts where there is a mention of Nazi
nazi_posts = comments_with_nazi_df['post_id'].unique()

#In cases where there is a comparison with Nazis or Hitler, identify the first comment 
#where the comparison is made
mins = comments_with_nazi_df.groupby('post_id')['RN'].idxmin()
first_nazi_comment = comments_with_nazi_df.loc[mins]

right_censored_posts = comments_df[comments_df.post_id.isin(nazi_posts) == False]
maxes = right_censored_posts.groupby('post_id')['RN'].idxmax()
final_comment = right_censored_posts.loc[maxes]

#combine the censored posts with those where a comparison is made
concatenated = pd.concat([first_nazi_comment,final_comment])

T = concatenated['RN']
E = concatenated['nazi_in_comment']

from lifelines import KaplanMeierFitter
kmf = KaplanMeierFitter()
kmf.fit(T, event_observed=E)
kmf.plot()

plt.xlim(0,2000);
plt.title("Reddit Post Lifespan Prior to Mention of Nazi or Hitler");
plt.xlabel("Comments")
plt.ylabel("Fraciton of Posts Without Mention of Hitler or Nazis")